Design model, input, output size, forward pass
<br> Construct loss and optimizer
<br> Training loop
- forward pass: comput prediction and loss
- backward pass: gradients
- update weights

In [3]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

step -1: prepare datasets

In [4]:
bc=datasets.load_breast_cancer()

In [5]:
X,y=bc.data,bc.target

In [7]:
n_samples,n_features=X.shape
n_samples

569

In [8]:
n_features

30

In [9]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=21)

Scale the features

In [10]:
sc=StandardScaler()

In [11]:
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)
X_train=torch.from_numpy(X_train.astype(np.float32))
X_test=torch.from_numpy(X_test.astype(np.float32))
y_train=torch.from_numpy(y_train.astype(np.float32))
y_test=torch.from_numpy(y_test.astype(np.float32))


changing row to column vector. we only have row right now

In [13]:
y_train=y_train.view(y_train.shape[0],1)
y_test=y_test.view(y_test.shape[0],1)

Model

In [16]:
class LogisticRegression(nn.Module):
    def __init__(self,n_input_features):
        super(LogisticRegression,self).__init__()
        self.linear=nn.Linear(n_input_features,1)
    def forward(self,x):
        y_predicted=torch.sigmoid(self.linear(x))
        return y_predicted

In [17]:
model = LogisticRegression(n_features)

In [18]:
learning_rate=0.01

In [24]:
criterion=nn.BCELoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

In [25]:
n_epochs=100

In [27]:
for epoch in range(n_epochs):
    #forward pass
    y_predicted=model(X_train)
    #loss calculation
    loss=criterion(y_predicted,y_train)
    #backward pass
    loss.backward()
    #update
    optimizer.step()
    #zero gardients
    optimizer.zero_grad()
    
    if (epoch +1) %10==0:
        print(f'{epoch +1 }, loss ={loss.item():.4f}')

10, loss =0.5216
20, loss =0.4450
30, loss =0.3937
40, loss =0.3566
50, loss =0.3285
60, loss =0.3062
70, loss =0.2882
80, loss =0.2732
90, loss =0.2604
100, loss =0.2495


We are done here, we don't need gardients so no_grad

In [29]:
with torch.no_grad():
    y_predicted=model(X_test)
    y_predicted_cls=y_predicted.round()
    acc=y_predicted_cls.eq(y_test).sum()/float(y_test.shape[0])
    print(f'accuracy={acc:.4f}')
    

accuracy=0.9386
